# Lab 3
## Data Structures & Algorithms
### Thursday, 22 February 2024

## Today

* [Coding workflows](#workflows)
* [Dynamic web apps with flask](#dynamic)
* [Another flask example](#example)
* [Deploying your flask app](#deployment)
* [Exercises](#exercises)

## Refresher on coding workflows <a class="anchor" id="workflows"></a>

### Virtual environments

Virtual environments help you isolate the Python (or R) versions and packages that you need for a certain project from those you might need from a different one. When you create a new virtual environment using the `conda create -n env_name ...` command, miniconda automatically creates a new directory with the name you have chosen (in this case it would be `env_name`) inside miniconda's own directory structure. By default, this is in your `Users` folder or equivalent (e.g. for me the path to the new directory in this example would be `/Users/lenamangold/miniconda3/envs/env_name`).

Once you have created a virtual environment, you can completely ignore this folder - it's where conda installs Python & packages; **do not** create your project directories **inside** this miniconda folder structure. Instead, keep project directories in your normal folder structure; to use the conda environment (e.g. to run a jupyter notebook or a flask app), run `conda activate env_name` on the command line after having navigated into the desired folder using `cd`.

### Jupyter notebooks vs Python modules (and IDEs)
Jupyter notebooks (files with the `.ipynb` extension) are best for code exploration and to display code alongside markdown. While it is possible with some IDEs (e.g. some versions of Pycharm) to run Jupyter notebooks *inside* the IDE, this sometimes doesn't work very well and I would recommend running them from the command line.

Python modules (files with the `.py` extension) are where you write the functions and classes that will be reused later on (e.g. as part of a data science project or a web app, or even in a Jupyter notebook). IDEs like PyCharm can help you with things like testing, debugging, documenting, and refactoring of your functions and classes.

## Dynamic web apps with flask <a class="anchor" id="dynamic"></a>

Unlike static websites (which look the same to any user who visits them), dynamic websites can change their content based on the user and their input. With **any** website, it gets displayed to you after you enter a URL that triggers the browser to send a request to the web server where the website is hosted and which then sends back an HTML file (plus whatever else is necessary) to display the webpage.

**Static websites** are built by creating fixed HTML (+ CSS, JS) files ('client-side' languages), which are returned by the hosting server and then displayed in the browser. They are useful for small websites (portfolios, personal websites), since they are not easy to scale or personalise. But they are relatively straightforward to build and they are faster than dynamic websites.

**Dynamic websites** are built at the time they are requested from the server; the HTML files are created in a way that is 'personalised' (e.g. because of some user characteristics or user input). They are written in 'server-side' languages (e.g. PHP, Python) PLUS client-side languages. They are easier to update (you don't have to change every static HTML file), scalable, and can be personalised.



## Another flask example <a class="anchor" id="example"></a>

Let's create a new flask app, this time with a slightly more advanced user input and some more functionality: a basic calculator.

* create a new project, e.g. called `calculator_app`
* in the command line, navigate into this new project directory by running `cd calculator_app` (if you are currently located in the parent directory, otherwise you have to `cd` to the entire directory path). Navigating into the `calculator_app` folder is necessary because this is where you will create your main app module `flask_app.py` (and your `templates` folder).
* IMPORTANT: today, it should be called `flask_app.py` rather than anything else. We will see later why! Once you have created your app module, you will run the flask run command in the command line - this only works if you are located in the correct folder that contains the main module file.
* in the command line, activate the flask app environment you created in the last lab (normally we create a new environment for each project, but this is just another very small example, so we can stick with this one), by running `conda activate ...` where you replace the dots with your environment name. To remind yourself of the virtual environments that you already have created, you can run `conda env list` and it will display a list of all created environments.
* open the project in PyCharm and create a `flask_app.py` file in the root directory of the project (it should be saved inside the `calculator_app` folder)
* now insert the following code into your `flask_app.py` file (you will need the `render_template` and `request` methods later, so make sure to include them in the import statement):

```python
from flask import Flask, render_template, request

app = Flask(__name__)  # create the instance of the flask class


@app.route('/')
def home():
    return 'Home page'
```

* let us change our home page to show the result of a calculation. Replace the `home` function by the following:

```python
@app.route('/')
def home():
    value1 = 3
    value2 = 4
    result = value1 + value2
    return str(result)
```

* make your app dynamic, by letting the user pass variables to the URL:

```python
@app.route('/<value1>_<value2>')
def home(value1, value2):
    value1 = float(value1)
    value2 = float(value2)
    return value1 + value2
```

* next, let the user decide the operation that they want to use (for now, only allowing summation):

```python
@app.route('/<value1>_<operation>_<value2>')
def home(value1, value2, operation):
    value1 = float(value1)
    value2 = float(value2)
    if operation == 'addition':
        return str(value1 + value2)
    else:
        return 'Operation must be "addition"'
```

* now add another dynamic element to the app: users should be able to enter their values into fields on the app and then press a button to make the calculation. For this we need to create something called a **form element**, the inputs of which Python will send back to the server to change your variables (through the `POST` action). First, we use a template instead of creating our HTML in the `flask_app.py` file. Create a `templates` folder inside your project root and create an `index.html` file within the `templates` folder. Copy the following code into the empty `index.html` file:

```html
<!DOCTYPE html>
<html >
<head>
  <title>Flask Calculator</title>
</head>

<body>
	<h1>Calculator</h1>

    <form action="{{ url_for('calculate')}}" method="post">
        <input type="text" name="value1" placeholder="Enter the first number" required="required" />
        <input type="text" name="value2" placeholder="Enter the second number" required="required" />
        <input type="text" name="operation" placeholder="addition" required="required" />
        <button type="submit">Calculate</button>
    </form>

   <br>

   {{ printed_result }}

</body>
</html>
```

* let us take a look at what this is doing. You are creating an HTML form element with the following parts:
    * the `action` parameter is the URL that the form data is sent to once the user clicks the button with the type `submit`; in flask, the `url_for` function generates to URL to a particular view of your page (in this case the `/calculate` view)
    * the `method` parameter means that the data that a user puts into this form will be sent to the server with the POST method (we will also need to specify this in our route)
    * when a user presses the `submit` button of this form (which has the text 'Calculate' on it), the data that a user has put into the form gets sent to the server, which does some operations with it and then sends an HTML back to the web browser (in whatever you have defined in your code). In particular, the `name` tags in the different elements represent how this data gets sent to the server.
* we now also need to change our `flask_app.py` file so that the server knows how to deal with this form data and execute the calculation. Copy the following code into your `flask_app.py` file, replacing the entire `home` function you wrote before.

```python
@app.route('/')
def home():
    return render_template('index.html')


@app.route('/calculate', methods=['POST'])  # associating the POST method with this route
def calculate():

    # using the request method from flask to request the values that were sent to the server through the POST method
    value1 = request.form['value1']
    value2 = request.form['value2']
    operation = str(request.form['operation'])

    # convert the input to floating points
    value1 = float(value1)
    value2 = float(value2)

    if operation == 'addition':
        return render_template('index.html', printed_result=str(value1 + value2))
    else:
        return render_template('index.html', printed_result='Operation must be "addition"')
```

* here, we have used flask's `request` method, to retrieve the elements that we have defined in the HTML form (i.e. the text elements that we have given the `name` tag in the form)

## How to deploy your flask app? <a class="anchor" id="deployment"></a>

* so far, our apps have only been running 'locally'
* to publish them to the internet, you need to **deploy** your app to a hosting platform (so that it is no longer hosted on your local machine, but on a hosting platform where your code will be constantly running at some domain)
* we will choose the hosting platform called [pythonanywhere](https://www.pythonanywhere.com/), since it has a completely free option; there are many options for deployment, but most of them are not free (or have a free version, but require a credit card to sign up)

### Create your web app on pythonanywhere

* create a free PythonAnywhere account and log in
* go to the Web menu item and press Add new web app
* click next, then click on the Flask option and finally, choose the latest version of Python in the list
* you will now see the pythonanywhere project path - just accept the default one by clicking Next again
* in your browser, visit https://yourusername.pythonanywhere.com (replacing `yourusername` with your actual pythonanywhere username)

### Upload your project

* go to Web menu and scroll down to the Code section - click on Go to directory next to where it says Source code (this is where we will upload our files, so that they can be hosted on pythonanywhere)
* this takes you to the Files section of pythonanywhere; delete the `flask_app.py` file that is there by default
* use the Upload a file button and upload your own `flask_app.py` file (it is important that it has this exact name, since pythonanywhere will be looking for a file with this name)
* on the left, under Directories, enter the name `templates` into the box and press New directory
* Upload the `index.html` file into this directory
* go back to the Web menu and click `Reload https://yourusername.pythonanywhere.com`

## Exercises <a class="anchor" id="exercises"></a>

### Exercise 1

Go through the steps in the [Another flask example](#example) section to create your first basic calculator app.

### Exercise 2

Go through the steps in the [How to deploy your flask app](#deployment) section to employ your app.

### Exercise 3

Now that you have your first deployed app, go back to PyCharm and continue working on the code for your calculator (to update the deployed website later, you can simply replace the files you have uploaded to pythonanywhere at the end!)

Extend the functionality of your calculator by enabling the user to do subtraction, multiplication and division on top of addition. Include a mechanism that prints a warning message when a different operation is added to the text field.

### Solution 3

The `calculate` function in your `flask_app.py` file should look something like this:

```python
@app.route('/calculate', methods=['POST'])  # associating the POST method with this route
def calculate():

    # using the request method from flask to request the values that were sent to the server through the POST method
    value1 = request.form['value1']
    value2 = request.form['value2']
    operation = str(request.form['operation'])

    # convert the input to floating points
    value1 = float(value1)
    value2 = float(value2)

    # do the calculation
    if operation == 'add':
        result = value1 + value2
    elif operation == 'subtract':
        result = value1 - value2
    elif operation == 'divide':
        result = value1 / value2
    elif operation == 'multiply':
        result = value1 * value2
    else:
        return render_template('index.html',
                               printed_result='Operation must be one of "add", "subtract", "divide", or "multiply".')

    return render_template('index.html', printed_result=str(result))
```

### Exercise 4

Now turn the text field that we have been using to type in the operation into a drop-down menu. Hint: Use the `<select>` form element from this [resource](https://www.w3schools.com/html/html_form_elements.asp); remember that the values of the form elements get sent to the server according to the `name` tag, so make sure that you set `name="operation"` in the HTML code for the drop-down.

### Solution 4

Your `index.html` file should look something like this:

```html
<!DOCTYPE html>
<html >
<head>
  <title>Flask Calculator</title>
</head>

<body>
	<h1>Calculator</h1>

    <form action="{{ url_for('calculate')}}" method="post">
        <input type="text" name="value1" placeholder="Enter the first number" required="required" />
        <input type="text" name="value2" placeholder="Enter the second number" required="required" />

        <label for="operation">Operation</label>
        <select id="operation" name="operation">
            <option value="add">Add</option>
            <option value="subtract">Subtract</option>
            <option value="divide">Divide</option>
            <option value="multiply">Multiply</option>
        </select>

        <button type="submit">Calculate</button>
    </form>

   <br>

   {{ printed_result }}

</body>
</html>
```

### Exercise 5

What kinds of problems could you run into with this calculator? Improve your code to deal with these problems by using the exception handling functionality described [here](https://www.w3schools.com/python/python_try_except.asp) (try the examples on this tutorial first, to understand how you can handle errors in Python).

Hint: Go through the lines of code in your `calculate` function and think about the types of thing that might go wrong. One of the problems is related to the type of the variable a user might put in the calculator, another is related to one of the operations.

### Solution 5

One problem might arise when you are trying to turn the values to floats, for example when the user puts in non-numeric text. Another is related to the zero division error. You can use the try

Your `calculate` function could now look like this (note that we have moved the check of the operation type out of the block of code that does the calculation for readability.

```python
@app.route('/calculate', methods=['POST'])  # associating the POST method with this route
def calculate():

    # using the request method from flask to request the values that were sent to the server through the POST method
    value1 = request.form['value1']
    value2 = request.form['value2']
    operation = str(request.form['operation'])

    # make sure the input is one of the allowed inputs (not absolutely necessary in the drop-down case)
    if operation not in ['add', 'subtract', 'divide', 'multiply']:
        return render_template('index.html',
                               printed_result='Operation must be one of "add", "subtract", "divide", or "multiply".')

    try:
        # convert the input to floating points
        value1 = float(value1)
        value2 = float(value2)

        # do the calculation
        if operation == 'add':
            result = value1 + value2
        elif operation == 'subtract':
            result = value1 - value2
        elif operation == 'divide':
            result = value1 / value2
        else:
            result = value1 * value2

        return render_template('index.html', printed_result=str(result))

    except ZeroDivisionError:
        return render_template('index.html', printed_result="You cannot divide by zero")

    except ValueError:
        return render_template('index.html', printed_result="Cannot perform operation with this input")
```

### Exercise 6

Now do some 'refactoring' of your code (code refactoring is when you restructure your code in a way that does not change the external behaviour but readability, scalability, and helps to prevent bugs):

* create a new python module called `helper.py` in your root directory into which you will write functions that you can then import into your main app module
* in `helper.py`, create a function which will perform your calculations, that takes as its input `value1`, `value2`, and `operation` and returns the calculated result
* import this function into the `flask_app.py` module, by writing `from helper import ...` where you replace `...` with your function name
* in the `calculate` function in `flask_app.py`, you can now call this function to get the result of the calculation
* EXTENSION: write another helper function in `helper.py` that handles the conversion of the input to type `float` and call this function in `flask_app.py` *before* calling the function that performs the operation. Wrap this function in a separate exception handler.

### Solution 6

The content of your `helper.py` file could look something like this:

```python
# create helper functions for calculations

def perform_calculation(value1: float, value2: float, operation: str) -> float:
    """
    Perform a mathematical operation on two values.

    Parameters:
        value1 (float): The first value.
        value2 (float): The second value.
        operation (str): The operation to perform. Can be 'add', 'subtract', 'divide', or 'multiply'.

    Returns:
        float: The result of the operation.

    Raises:
        ZeroDivisionError: If attempting to divide by zero.
    """
    if operation == 'add':
        result = value1 + value2
    elif operation == 'subtract':
        result = value1 - value2
    elif operation == 'divide':
        result = value1 / value2
    else:
        result = value1 * value2

    return result


def convert_to_float(value1: str, value2: str) -> tuple[float, float]:
    """
    Convert two strings to floating point numbers.

    Parameters:
        value1 (str): The first value to convert.
        value2 (str): The second value to convert.

    Returns:
        tuple[float, float]: A tuple containing the converted float values of value1 and value2.

    Raises:
        ValueError: If either value1 or value2 cannot be converted to a float.
    """

    float_value1 = float(value1)
    float_value2 = float(value2)

    return float_value1, float_value2
```

Your `flask_app.py` file should now look like this:

```python
from flask import Flask, render_template, request

from helper import perform_calculation, convert_to_float

app = Flask(__name__)  # create the instance of the flask class


@app.route('/')
def home():
    return render_template('index.html')


@app.route('/calculate', methods=['POST'])  # associating the POST method with this route
def calculate():

    # using the request method from flask to request the values that were sent to the server through the POST method
    value1 = request.form['value1']
    value2 = request.form['value2']
    operation = str(request.form['operation'])

    # make sure the input is one of the allowed inputs (not absolutely necessary in the drop-down case)
    if operation not in ['add', 'subtract', 'divide', 'multiply']:
        return render_template('index.html',
                               printed_result='Operation must be one of "add", "subtract", "divide", or "multiply".')

    try:
        value1, value2 = convert_to_float(value1=value1, value2=value2)
    except ValueError:
        return render_template('index.html', printed_result="Cannot perform operation with this input")

    try:
        result = perform_calculation(value1=value1, value2=value2, operation=operation)
        return render_template('index.html', printed_result=str(result))

    except ZeroDivisionError:
        return render_template('index.html', printed_result="You cannot divide by zero")


```

### Exercise 7

Try calculating $4.9 - 4.845$ with your calculator. What do you see? Include something in your code that will return the value that you would expect.

### Solution 7

The result we get is 0.055000000000000604 instead of the expected 0.55. This happens because the floating point cannot be represented exactly as a binary fraction. The decimal floating-point number is therefore approximated by the binary floating-point number that is *actually* stored in the machine - this is causing a small **round-off error**.

If you want to solve this in your calculator, you can simply use the built-in `round` function, which you can include by passing `printed_result=str(round(result, 3))` ro the `render_template` function, instead of `printed_result=str(result)`.